In [26]:
import pandas as pd
import numpy as np
import gc
from collections import Counter
import csv
import math
import gzip
import pathlib
import os

from nltk.tokenize.casual import TweetTokenizer

from stan_data_imports.social.social_apis.twitter import TwitterAPI
from stan_data_imports.social.twitter_import import create_twitter_df

In [27]:
cwd = pathlib.Path.cwd()
repo_dir = cwd.parent
dataset_dir = repo_dir / "datasets" / "semeval"

# Load task A
subtask_dir = dataset_dir / "Subtasks_CE"

In [5]:
data = []
for file in os.listdir(subtask_dir):
    if file[-4:] == '.txt':
        with open(subtask_dir / file) as f:
            print(file)
            data += f.readlines()
data = [line.replace('\n','').replace(' ','').split('\t') for line in data]

df_a = pd.DataFrame(data, columns=["ID", "Topic", "Sentiment"])
df_a = df_a[["ID", "Topic", "Sentiment"]]
df_a["Sentiment"] = df_a["Sentiment"].astype(np.int)
print(df_a.shape)
df_a.drop_duplicates(subset=["ID"], inplace=True)
df_a = df_a[df_a["Sentiment"] != 'offtopic']
df_a["Sentiment"] = df_a["Sentiment"].replace(-2,-1).replace(2,1)
df_a["Sentiment"] = df_a["Sentiment"].replace(-1,"Negative").replace(1, "Positive").replace(0, "Neutral")
print(df_a.shape)

df_a.index = df_a["ID"]
df_a.drop("ID", 1, inplace=True)
map_a = df_a.to_dict(orient='index')

twitter-2016dev-CE.txt
twitter-2016devtest-CE.txt
twitter-2016test-CE.txt
twitter-2016train-CE.txt
(30632, 3)
(30432, 3)


C:\Users\usherwoodpe\AppData\Roaming\Python\Python36\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
df_a["Sentiment"].value_counts()

Positive    13867
Neutral     12898
Negative     3667
Name: Sentiment, dtype: int64

In [7]:
api = TwitterAPI()
tweets_raw = api.fortify_twitter_tweets_batch(tweet_ids=df_a.index.tolist())

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=639026260887359489%2C639112915140911105%2C638958465268994048%2C635153350254837760%2C636686318530129920%2C629983374564655104%2C638360406008791040%2C636167517354876928%2C638536973842554880%2C637614000461729792%2C634682553492996096%2C639119941128585216%2C632111203763027968%2C635266768798220289%2C628388898163404800%2C639341324777340928%2C632209682677940224%2C634278703824945152%2C632338922090143745%2C641167597170388992%2C635427775172624384%2C632189973614170112%2C674760248284938241%2C671573150971248640%2C676269663320338436%2C670821762473533440%2C671959175166038016%2C676263010638233600%2C669614426685882368%2C660493289519362048%2C676632130760392705%2C660512979377455104%2C680723269016784896%2C660514655799001088%2C674734715035758592%2C676912784177807360%2C666417339915628544%2C670683455500390400%2C675264733256261633%2C676124729900683264%2C677170113255186432%2C

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=669132223832977408%2C676522142763237376%2C671198301669208064%2C678024857909522432%2C661102247590297600%2C677564327336026112%2C665352318611808258%2C668212490287869952%2C681115613297815553%2C675106220001370112%2C677263906650365953%2C667730025630867456%2C664487997149024256%2C674176348592689156%2C674168854394904576%2C666578060356001792%2C675008585827540992%2C660571582650281984%2C671044390467608576%2C667196286483959808%2C664088967491416064%2C664426748692537344%2C681013896359112704%2C663902636131774464%2C668377673224220672%2C662033551181914112%2C673600106873151489%2C674333610846109697%2C675125335613169668%2C681575886882258945%2C665239080083566592%2C679250307931635712%2C677576167927513088%2C666232693127045120%2C665945515088068610%2C681016713656287232%2C675717600337264640%2C679710770469285888%2C681190124965740545%2C666808084484874240%2C669493841779695617%2C

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=633717676238598146%2C635238536657117185%2C634900107570221056%2C635315886031806464%2C636865904979144704%2C635580989251682304%2C636177465283616768%2C636362190249857025%2C634813883979550720%2C634194755765825536%2C638880136926990336%2C634105293425782785%2C635514064937058304%2C634420935987232770%2C641442257204346880%2C630948990637375488%2C635770184955916288%2C637663053438279680%2C633503403541446656%2C637450663048441856%2C635904030452006913%2C640971550334787584%2C637098907651149824%2C635603175425179648%2C640973991113531392%2C635959188678533120%2C641474163610419200%2C635959075663147008%2C635631102405816321%2C636176735701233664%2C631119350435586048%2C634373635046809600%2C635935251282440192%2C637765956375347200%2C637666672992718849%2C635463616356065280%2C634163261513068546%2C636624208446005249%2C635962413691465733%2C634095148033527808%2C635959160060964865%2C

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=641056534718496768%2C641007024797564929%2C637304055098941440%2C630001075777609728%2C628743845870604288%2C638908153506230272%2C639949540532060161%2C641570475022372864%2C625703427381837824%2C638701657971392512%2C625910792509001728%2C633434707896762368%2C630428491763306496%2C638885450795589632%2C638429746557255680%2C629822713666928640%2C623739595591544834%2C626548149771993088%2C639210552103931904%2C638530211148120064%2C633343448431624192%2C641268447805075456%2C633716235369357312%2C637074952894738432%2C621671339569598469%2C631109986911920128%2C638008134452314112%2C641401844988821504%2C633851431611691008%2C639228268110086144%2C631317480544292864%2C633474523329875968%2C633810133223538689%2C627711198805536768%2C629625106952757249%2C636262361775648768%2C626197453130219520%2C624601038486528000%2C641602348616404992%2C639680839278718976%2C637275452298629120%2C

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=638494049335246852%2C640028671839834112%2C641446206586155008%2C640251769268891649%2C638793275697201152%2C641218455753097216%2C637471213913112579%2C641416826354798593%2C641098536147357696%2C641412899668389888%2C636760839190503424%2C639255970154070016%2C641577139632910336%2C640855853990481921%2C638136656932040709%2C639788496543203328%2C637262203612217344%2C638763340483928064%2C640869494789902336%2C639067142873329665%2C637250275888107523%2C639070104123129856%2C640485442249465856%2C637303639929819136%2C638548727104274432%2C639583855574417408%2C639959715938570242%2C640363430873067520%2C641395951278538752%2C638863098632343552%2C639257300054724609%2C637050316551274496%2C638578826533044224%2C637396597077250048%2C639658363802906625%2C641076323520258048%2C637053876345159680%2C638142009341947904%2C638441630438060032%2C638871588172079104%2C641432698456772608%2C

...

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=636402710632206341%2C640199927822426112%2C641626419580174336%2C637730954824523776%2C638013735496454144%2C635731799813951488%2C637364985446817792%2C635731354588573696%2C639956129980657665%2C638135679541624832%2C637872593194782720%2C639771593951047681%2C637163496145354752%2C636025403920941057%2C638873189821640704%2C638009918050709504%2C640500530746380288%2C635867786527412224%2C636292312822337536%2C639557797684768768%2C640089919747063808%2C639894860150013952%2C636524686147485696%2C637416656172486656%2C640968165569228800%2C637371406699749376%2C635909607613841408%2C638390131515387904%2C638570122702127104%2C640913970166124544%2C635603138544734208%2C637579649829208064%2C640818654448340992%2C639569558106447872%2C636212932276260864%2C638813358389653508%2C635623870171598849%2C640633870128386049%2C640971999607713792%2C637901642709340161%2C635221435917692928%2C

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=672250085816844288%2C678609996607823872%2C673940440916369413%2C673084060097363969%2C681677184080162816%2C671768047477202944%2C671518947225694209%2C676558815224045572%2C676203930615812096%2C677336105658793985%2C681690885076201472%2C672502650831671296%2C674376052618174464%2C675826669332992000%2C675481696062603264%2C681904976860327936%2C676980451647594499%2C678377263335870464%2C677618338441244672%2C679330432962289665%2C681373775150100480%2C681602615365337088%2C678695958352412672%2C676668722883526656%2C679372748666351616%2C681252811707912192%2C678850641373081600%2C678450314945531904%2C678386339881070592%2C677579762953535492%2C676711113468354560%2C679387827495301120%2C679495616746098688%2C679455098146304000%2C678814175259262976%2C678994987502149633%2C676064909415723008%2C677297424872837120%2C681145891789664257%2C676659299452624896%2C676318462034231297%2C

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=676358516567842816%2C676714386921230336%2C678344828891959296%2C675911713380818945%2C678462051363233794%2C676614201738137600%2C676273460369301508%2C678099281971146752%2C676650985859641344%2C678016775456489472%2C678810257112604672%2C676773252849524737%2C676350931869548545%2C677663904546336770%2C676447749370417153%2C675942303454928896%2C677448218217705473%2C680348092739235840%2C680998624126496768%2C676686975152349185%2C678694499711717377%2C679040705642262528%2C676882429966196736%2C676505148689108992%2C676681455603322880%2C678652087253733376%2C681811463745482752%2C676288941776109568%2C676571114835808256%2C678770214532685824%2C677557897887244289%2C678649255075147776%2C676480813278564353%2C678751009339023360%2C677909846927142912%2C680459244882231296%2C678820788611383296%2C678622965303549952%2C676012712980271108%2C678429569997807616%2C681298725554130945%2C

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=676516321035608065%2C675750898988154880%2C676082145140580352%2C676790666102312960%2C638411213890822144%2C676885147476619264%2C676371433375801344%2C637563389192105984%2C638612899716399104%2C678650195517681666%2C637506781942579200%2C678168377005891584%2C677502070694506497%2C640267367268806656%2C674932364099854337%2C640519153233715200%2C674865716793741312%2C641401301318897665%2C676067565244915712%2C678757319962976256%2C641314094444888064%2C676062430863204352%2C641135498287837184%2C675742753372831744%2C641339380364251136%2C681123455262191620%2C674606861207908352%2C675063884777148416%2C681503382708850689%2C679411898669006848%2C676009071649116160%2C641244603501309953%2C640865190808940544%2C681449367174299648%2C638114574156238848%2C681476407243874304%2C677322190769942528%2C640232928589381632%2C675244310070169600%2C678928911577841664%2C639257387614924800%2C

Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=641009206422847491%2C641411664538152960%2C641014106200387584%2C641024944554557440%2C641024610788601857%2C641024211713171456%2C641053565100929024%2C641023790252752897%2C641054703967711232%2C641040487655317505%2C641021454407049216%2C641015328508653568%2C641009530940342272%2C640593299733151745%2C641023634346246144%2C640341980749426688%2C640896497341571072%2C641036659669250048%2C641010486096588801%2C640330182029639681%2C641016755872264192%2C641044530347835392%2C641007298593333249%2C641049995479752704%2C641053855921401856%2C641053712509706244%2C641587199746641921%2C641026126597521408%2C641030410328276992%2C641120802499371008%2C640636168669786113%2C640701965517869057%2C640343175962460160%2C640617348748210176%2C641029750643904512%2C640361266742099969%2C640746566941310976%2C640366627767353344%2C640896713599811584%2C641013370603237376%2C641029934350254080%2C

In [8]:
twit_mentions = []
twit_users = []

for tweet in tweets_raw[0]:
    twit_mentions.append(api.parse_tweet_to_twitter_mention(tweet))
    
for user in tweets_raw[1]:
    twit_users.append(api.parse_user_to_twitter_user(user))
    
df_a_tweets = create_twitter_df(twit_mentions, twit_users)
df_a_tweets.head()

Total Records: 15289


,Tweet ID,Domain,Source,Url,Author ID,Date (GMT),Date (Local),Date (Local - Zone),Snippet,Sentiment,...,Bio,Profile Picture URL,Follower Count,Profile Picture,Verified,Number of Statuses,Date Created,Status Merged,Listed Count,Friends Count
0,640755399042449408,twitter.com,TwitterAPI,https://twitter.com/statuses/640755399042449408,twitter.com24801889,2015-09-07 05:16:06+00:00,None,None,3rd: The Weeknd sounds just like Michael Jackson,Not Found,...,I’m a Writer ✨ & Photographer 📷 That’s cool ri...,https://pbs.twimg.com/profile_images/111252355...,622,https://pbs.twimg.com/profile_images/111252355...,False,37163,2009-03-17 00:08:17+00:00,[],15,493
1,641104014172524545,twitter.com,TwitterAPI,https://twitter.com/statuses/641104014172524545,twitter.com325878156,2015-09-08 04:21:22+00:00,None,None,"I spent all weekend in the sun, and I'm still ...",Not Found,...,"I am here to live out loud, but only if I don’...",https://pbs.twimg.com/profile_images/112482305...,133,https://pbs.twimg.com/profile_images/112482305...,False,5473,2011-06-29 01:02:05+00:00,[],4,168
2,639131453649985541,twitter.com,TwitterAPI,https://twitter.com/statuses/639131453649985541,twitter.com63640242,2015-09-02 17:43:07+00:00,None,None,It is too hot. I was waking around campus w my...,Not Found,...,well rounded like cursive ➰,https://pbs.twimg.com/profile_images/112074846...,1008,https://pbs.twimg.com/profile_images/112074846...,False,34956,2009-08-07 04:39:35+00:00,[],11,506
3,638594008437104640,twitter.com,TwitterAPI,https://twitter.com/statuses/638594008437104640,twitter.com1428368701,2015-09-01 06:07:30+00:00,None,None,Jay Z 'pays tribute to Michael Jackson as he j...,Not Found,...,#Blogger #Philosopher #MusicArtist #Lover #Nig...,https://pbs.twimg.com/profile_images/938535784...,3962,https://pbs.twimg.com/profile_images/938535784...,False,319238,2013-05-14 17:09:45+00:00,[],64,4432
4,639395430472790016,twitter.com,TwitterAPI,https://twitter.com/statuses/639395430472790016,twitter.com3139373229,2015-09-03 11:12:04+00:00,None,None,"Remembering Michael Jackson\nThursday, Septemb...",Not Found,...,0745999222,https://pbs.twimg.com/profile_images/614091779...,23,https://pbs.twimg.com/profile_images/614091779...,False,820,2015-04-06 06:30:59+00:00,[],0,144


In [9]:
df_a_tweets["Sentiment"] = df_a_tweets["Tweet ID"].apply(lambda e: map_a[e]['Sentiment'])
df_a_tweets["Topic"] = df_a_tweets["Tweet ID"].apply(lambda e: map_a[e]['Topic'])

In [10]:
df_a_tweets["Sentiment"].value_counts()

Positive    7478
Neutral     5928
Negative    1883
Name: Sentiment, dtype: int64

In [11]:
df = df_a_tweets
df

,Tweet ID,Domain,Source,Url,Author ID,Date (GMT),Date (Local),Date (Local - Zone),Snippet,Sentiment,...,Profile Picture URL,Follower Count,Profile Picture,Verified,Number of Statuses,Date Created,Status Merged,Listed Count,Friends Count,Topic
0,640755399042449408,twitter.com,TwitterAPI,https://twitter.com/statuses/640755399042449408,twitter.com24801889,2015-09-07 05:16:06+00:00,None,None,3rd: The Weeknd sounds just like Michael Jackson,Neutral,...,https://pbs.twimg.com/profile_images/111252355...,622,https://pbs.twimg.com/profile_images/111252355...,False,37163,2009-03-17 00:08:17+00:00,[],15,493,michaeljackson
1,641104014172524545,twitter.com,TwitterAPI,https://twitter.com/statuses/641104014172524545,twitter.com325878156,2015-09-08 04:21:22+00:00,None,None,"I spent all weekend in the sun, and I'm still ...",Neutral,...,https://pbs.twimg.com/profile_images/112482305...,133,https://pbs.twimg.com/profile_images/112482305...,False,5473,2011-06-29 01:02:05+00:00,[],4,168,michaeljackson
2,639131453649985541,twitter.com,TwitterAPI,https://twitter.com/statuses/639131453649985541,twitter.com63640242,2015-09-02 17:43:07+00:00,None,None,It is too hot. I was waking around campus w my...,Neutral,...,https://pbs.twimg.com/profile_images/112074846...,1008,https://pbs.twimg.com/profile_images/112074846...,False,34956,2009-08-07 04:39:35+00:00,[],11,506,michaeljackson
3,638594008437104640,twitter.com,TwitterAPI,https://twitter.com/statuses/638594008437104640,twitter.com1428368701,2015-09-01 06:07:30+00:00,None,None,Jay Z 'pays tribute to Michael Jackson as he j...,Positive,...,https://pbs.twimg.com/profile_images/938535784...,3962,https://pbs.twimg.com/profile_images/938535784...,False,319238,2013-05-14 17:09:45+00:00,[],64,4432,michaeljackson
4,639395430472790016,twitter.com,TwitterAPI,https://twitter.com/statuses/639395430472790016,twitter.com3139373229,2015-09-03 11:12:04+00:00,None,None,"Remembering Michael Jackson\nThursday, Septemb...",Positive,...,https://pbs.twimg.com/profile_images/614091779...,23,https://pbs.twimg.com/profile_images/614091779...,False,820,2015-04-06 06:30:59+00:00,[],0,144,michaeljackson
5,639520546552684545,twitter.com,TwitterAPI,https://twitter.com/statuses/639520546552684545,twitter.com224715473,2015-09-03 19:29:14+00:00,None,None,"On this day (September 03, 2009), Michael Jack...",Positive,...,https://pbs.twimg.com/profile_images/111503380...,749,https://pbs.twimg.com/profile_images/111503380...,False,47452,2010-12-09 17:54:08+00:00,[],2,1623,michaeljackson
6,638818244434923520,twitter.com,TwitterAPI,https://twitter.com/statuses/638818244434923520,twitter.com29184873,2015-09-01 20:58:32+00:00,None,None,@autoengmike @Minutemaniac @ImStillARebel @col...,Positive,...,https://pbs.twimg.com/profile_images/953576936...,581,https://pbs.twimg.com/profile_images/953576936...,False,54307,2009-04-06 11:46:44+00:00,[],45,477,michaeljackson
7,641016681234628612,twitter.com,TwitterAPI,https://twitter.com/statuses/641016681234628612,twitter.com271145751,2015-09-07 22:34:20+00:00,None,None,when i was in 4th grade my sister convinced me...,Positive,...,https://pbs.twimg.com/profile_images/112512819...,199,https://pbs.twimg.com/profile_images/112512819...,False,5506,2011-03-23 23:20:00+00:00,[],0,169,michaeljackson
8,638425960111546368,twitter.com,TwitterAPI,https://twitter.com/statuses/638425960111546368,twitter.com198253370,2015-08-31 18:59:44+00:00,None,None,@Pearsey I actually believed that his 2nd cous...,Neutral,...,https://pbs.twimg.com/profile_images/864722970...,333,https://pbs.twimg.com/profile_images/864722970...,False,21940,2010-10-03 19:52:10+00:00,[],8,716,michaeljackson
9,639090903169142786,twitter.com,TwitterAPI,https://twitter.com/statuses/639090903169142786,twitter.com924782179,2015-09-02 15:01:59+00:00,None,None,Let us dream of tomorrow where we can truly lo...,Positive,...,https://pbs.twimg.com/profile_images/987710387...,28370,https://pbs.twimg.com/profile_images/987710387...,False,5229,2012-11-04 06:19:56+00

In [12]:
def my_round(x, base=.05, prec=2):
    return round(base * round(float(x) / base), prec)


def split_dataset_into_even_class_distributions(X_data, Y_data, train_size, test_size, mini_batch_size=32):
    """
    Split a dataset into train, validate, and test sets that have the same distribution of classes as the original data.
    Also this function ensures the resulting datasets are multiples of the mini_batch_size

    :param X_data: Numpy 2d array or pandas dataframe, each row is a record
    :param Y_data: Pandas Series, the categorial class labels
    :param train_size: float 0-1, quantity of the original dataset to be in the train set (val is created from the residual)
    :param test_size: float 0-1, quantity of the original dataset to be in the test set (val is created from the residual)
    :param mini_batch_size: Int, the size of the mini batches to ensure each dataset is a multiple of that values

    :return: 6 data sets of X and Y
    """

    dist = Counter(val for val in Y_data)
    print('Total class distribution:', dict(dist))

    Y_data.reset_index(drop=True, inplace=True)
    X_data.reset_index(drop=True, inplace=True)

    train_ix = []
    validate_ix = []
    test_ix = []
    for cls in dist.keys():
        num_train = my_round(dist[cls] * train_size, base=mini_batch_size)
        num_test = my_round(dist[cls] * test_size, base=mini_batch_size)

        cls_targets = Y_data[Y_data == cls]

        full_ixs = np.random.choice(cls_targets.index, size=num_train, replace=False)

        train_ix += list(full_ixs)

        cls_targets = cls_targets[~cls_targets.index.isin(full_ixs)]

        full_ixs = np.random.choice(cls_targets.index, size=num_test, replace=False)

        test_ix += list(full_ixs)

        cls_targets = cls_targets[~cls_targets.index.isin(full_ixs)]

        validate_ix += list(cls_targets.index)

    X_train = X_data[train_ix]
    Y_train = Y_data[train_ix].reset_index(drop=True)

    X_validate = X_data[validate_ix]
    Y_validate = Y_data[validate_ix].reset_index(drop=True)

    X_test = X_data[test_ix]
    Y_test = Y_data[test_ix].reset_index(drop=True)

    print('Train class distribution:', dict(Counter(val for val in Y_train)))
    print('Validate class distribution:', dict(Counter(val for val in Y_validate)))
    print('Test class distribution:', dict(Counter(val for val in Y_test)))

    return X_train, Y_train, X_validate, Y_validate, X_test, Y_test


def parse_df_into_sets_by_tertiary_grouping(df, ter="ID", y="Score", x="text", train_prop=.7, test_prop=.2):

    ideal_train_size = math.floor(len(df)*train_prop)
    ideal_test_size = math.floor(len(df)*test_prop)
    ideal_val_size = len(df) - (ideal_train_size + ideal_test_size)

    title_avg_count = df[ter].value_counts().mean()

    titles = df[ter].unique().tolist()

    train_titles = []
    train_count = 0

    while train_count <= ideal_train_size - title_avg_count:
        title = np.random.choice(titles)
        titles.remove(title)
        train_titles.append(title)
        train_count += len(df[df[ter] == title])

    test_titles = []
    test_count = 0

    while test_count <= ideal_test_size - title_avg_count:
        title = np.random.choice(titles)
        titles.remove(title)
        test_titles.append(title)
        test_count += len(df[df[ter] == title]) 

    val_titles = titles
    val_count = len(df[df[ter].isin(val_titles)])

    print("Ideal train size is:", ideal_train_size, "whearas the actual is:", train_count)
    print("Ideal val size is:", ideal_val_size, "whearas the actual is:", val_count)
    print("Ideal test size is:", ideal_test_size, "whearas the actual is:", test_count)

    df_trn = df[df[ter].isin(train_titles)]
    df_val = df[df[ter].isin(val_titles)]
    df_tst = df[df[ter].isin(test_titles)]
    
    print("Train class distribution:", df_trn[y].value_counts())
    print("Val class distribution:", df_val[y].value_counts())
    print("Test class distribution:", df_tst[y].value_counts())

    return df_trn[x], df_trn[y], df_val[x], df_val[y], df_tst[x], df_tst[y]

def balance_classes(X_data, Y_data, min_class_value=None):
    
    kes = Y_data.value_counts().keys().tolist()
    
    min_class = Y_data.value_counts().keys()[-1]
    
    if not min_class_value:
        min_class_value = Y_data.value_counts().values[-1]
    
    all_ix = []
            
    for ke in kes:
        ke_indexes = Y_data[Y_data == ke].index.tolist()
        if len(ke_indexes) == min_class_value:
            all_ix += ke_indexes
        else:
            all_ix += list(np.random.choice(ke_indexes, size=min_class_value, replace=False))
        
    X_data = X_data[all_ix]
    Y_data = Y_data[all_ix]
    
    print("Y class distribution:", Y_data.value_counts())
    
    return X_data, Y_data

In [13]:
df["Sentiment"].value_counts()

Positive    7478
Neutral     5928
Negative    1883
Name: Sentiment, dtype: int64

In [14]:
# Unbalanced

X_train, Y_train, X_validate, Y_validate, X_test, Y_test = parse_df_into_sets_by_tertiary_grouping(df=df,
                                                                                                   ter="Topic",
                                                                                                   y="Sentiment",
                                                                                                   x="Snippet",
                                                                                                   train_prop=.4, 
                                                                                                   test_prop=.4)

N_CLASSES = len(Y_train.value_counts())
print(N_CLASSES, "classes")

df_trn = pd.DataFrame(np.array([Y_train, X_train]).T, columns=['label', 'text'])
df_val = pd.DataFrame(np.array([Y_validate, X_validate]).T, columns=['label', 'text'])
df_tst = pd.DataFrame(np.array([Y_test, X_test]).T, columns=['label', 'text'])

Ideal train size is: 6115 whearas the actual is: 6094
Ideal val size is: 3059 whearas the actual is: 3168
Ideal test size is: 6115 whearas the actual is: 6027
Train class distribution: Positive    2992
Neutral     2370
Negative     732
Name: Sentiment, dtype: int64
Val class distribution: Positive    1888
Neutral     1104
Negative     176
Name: Sentiment, dtype: int64
Test class distribution: Positive    2598
Neutral     2454
Negative     975
Name: Sentiment, dtype: int64
3 classes


In [15]:
for size in [100, 300, 1000]:
    df_trn.sample(n=size*N_CLASSES).to_csv(subtask_dir / ("fair_unbalanced_"+str(size)) / "train.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
    df_val.to_csv(subtask_dir / ("fair_unbalanced_"+str(size)) / "validate.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
    df_tst.to_csv(subtask_dir / ("fair_unbalanced_"+str(size)) / "test.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')

In [17]:
# Balanced
X_train, Y_train, X_validate, Y_validate, X_test, Y_test = parse_df_into_sets_by_tertiary_grouping(df=df,
                                                                                                   ter="Topic",
                                                                                                   y="Sentiment",
                                                                                                   x="Snippet",
                                                                                                   train_prop=.4, 
                                                                                                   test_prop=.4)

X_validate, Y_validate = balance_classes(X_data=X_validate, Y_data=Y_validate)
X_test, Y_test = balance_classes(X_data=X_test, Y_data=Y_test, min_class_value=1000)

df_val = pd.DataFrame(np.array([Y_validate, X_validate]).T, columns=['label', 'text'])
df_tst = pd.DataFrame(np.array([Y_test, X_test]).T, columns=['label', 'text'])

Ideal train size is: 6115 whearas the actual is: 6052
Ideal val size is: 3059 whearas the actual is: 3070
Ideal test size is: 6115 whearas the actual is: 6167
Train class distribution: Positive    2949
Neutral     2220
Negative     883
Name: Sentiment, dtype: int64
Val class distribution: Positive    1349
Neutral     1329
Negative     392
Name: Sentiment, dtype: int64
Test class distribution: Positive    3180
Neutral     2379
Negative     608
Name: Sentiment, dtype: int64
Y class distribution: Positive    392
Negative    392
Neutral     392
Name: Sentiment, dtype: int64


ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
for size in [100, 300, 1000]:
    X_train_crnt, Y_train_crnt = balance_classes(X_data=X_train, Y_data=Y_train, min_class_value=size)
    df_trn = pd.DataFrame(np.array([Y_train_crnt, X_train_crnt]).T, columns=['label', 'text'])
    df_trn.to_csv(subtask_dir / ("fair_balanced_"+str(size)) / "train.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
    df_val.to_csv(subtask_dir / ("fair_balanced_"+str(size)) / "validate.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
    df_tst.to_csv(subtask_dir / ("fair_balanced_"+str(size)) / "test.csv", index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')

In [ ]:
#TEST

df = pd.read_csv(subtask_dir / ("fair_unbalanced_"+str(300)) / "train.csv")
df['label'].value_counts()

In [28]:
#FIXUP

folders = ["fair_balanced_100", "fair_balanced_300", "fair_balanced_1000", "fair_unbalanced_100", "fair_unbalanced_300", "fair_unbalanced_1000"]

for folder in folders:
    files = os.listdir(subtask_dir / folder)
    for file in files:
        df = pd.read_csv(subtask_dir / folder / file)
        print(subtask_dir / folder / file)
        #df["label"] = df["label"].replace(-1,"Negative").replace(1, "Positive").replace(0, "Neutral")
        print(df['label'].value_counts())
        #df.to_csv(subtask_dir / folder / file, index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')

C:\Users\usherwoodpe\Documents\bibliotecas\low_shot_tl\datasets\semeval\Subtasks_CE\fair_balanced_100\test.csv
Negative    1000
Positive    1000
Neutral     1000
Name: label, dtype: int64
C:\Users\usherwoodpe\Documents\bibliotecas\low_shot_tl\datasets\semeval\Subtasks_CE\fair_balanced_100\train.csv
Negative    100
Positive    100
Neutral     100
Name: label, dtype: int64
C:\Users\usherwoodpe\Documents\bibliotecas\low_shot_tl\datasets\semeval\Subtasks_CE\fair_balanced_100\validate.csv
Positive    546
Negative    546
Neutral     546
Name: label, dtype: int64
C:\Users\usherwoodpe\Documents\bibliotecas\low_shot_tl\datasets\semeval\Subtasks_CE\fair_balanced_300\test.csv
Negative    1000
Positive    1000
Neutral     1000
Name: label, dtype: int64
C:\Users\usherwoodpe\Documents\bibliotecas\low_shot_tl\datasets\semeval\Subtasks_CE\fair_balanced_300\train.csv
Positive    300
Negative    300
Neutral     300
Name: label, dtype: int64
C:\Users\usherwoodpe\Documents\bibliotecas\low_shot_tl\datasets